# Assignment 4: Evaluating Search Engines

For this assignment, we leave aside the code we developed so far, and look into the more general issue of how to evaluate and compare different search engines. The ultimate test for any Information Retrieval system is how well it is able to satisfy the information needs of users.

# Cohen's Kappa

Our evaluation will involve the calculation of [Cohen's kappa](https://en.wikipedia.org/wiki/Cohen's_kappa) to quantify the degree to which two human assessors agree or disagree on whether results are considered relevant or not. To calculate Cohen's kappa, we are going to use the [scikit-learn library](http://scikit-learn.org/stable/):

In [1]:
! pip install --user scikit-learn

In [2]:
from sklearn.metrics import cohen_kappa_score

This library expects relevance assessments as lists of elements where `1` stands for _relevant_ and `0` stands for _not relevant_, for example like this:

In [3]:
a1=[1,0,1,0,1,0,1,0]

This list means that the first document was assessed to be relevant, the second to be not relevant, the third to be relevant etc.

We need two assessments in order to calculate Cohen's kappa, so let's make another exemplary list that only differs on the last element:

In [4]:
a2=[1,0,1,0,1,0,1,1]

We can now invoke the library as follows to calculate the agreement between the two:

In [5]:
cohen_kappa_score(a1, a2)

0.75

This value represents high agreement. We can reach maximal agreement if the two assessments are identical:

In [6]:
cohen_kappa_score(a1, a1)

1.0

Now, let's see what happens for a third assessment that differs on three positions with the first one (the three last positions):

In [7]:
a3=[1,0,1,0,1,1,0,1]

cohen_kappa_score(a1, a3)

0.25

We get a smaller but still positive value, because these two assessments still mostly agree. If we make a further example that differs on 6 of the 8 positions, we get the following result:

In [8]:
a4=[1,0,0,1,0,1,0,1]

cohen_kappa_score(a1, a4)

-0.5

The score is now negative, because the two differ on more positions than they agree. The agreement is in fact less than what you would expect to occur just by chance. We get the maximal disagreement if we define a fifth example that disagrees on all positions:

In [9]:
a5=[0,1,0,1,0,1,0,1]

cohen_kappa_score(a1, a5)

-1.0

Be aware that the kappa score cannot be calculated if you have only `1`s or only `0`s:

In [10]:
a6=[1,1,1,1,1,1,1,1]
a7=[1,1,1,1,1,1,1,1]

cohen_kappa_score(a6, a7)

/home/spawn_delta/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


nan

And in the case of a highly skewed set (either vast majority of agreements on `1` or vast majority of agreements on `0`), the kappa score can be counter-intuitive:

In [11]:
a8=[1,1,1,1,1,1,0,1]
a9=[1,1,1,1,1,1,1,0]

cohen_kappa_score(a8, a9)

-0.1428571428571428

Now that we understand how this function works, we will apply it below for our specific evaluation.

# Results and Assessments

Next, we will define some auxilary code to deal with lists of URLs from search engines and associated relevance assessments. We will encode result lists like this:

In [12]:
urls = [
    'https://en.wikipedia.org/wiki/Information_retrieval/',  # 1st result
    'http://www.dictionary.com/browse/information',          # 2nd result
    'https://nlp.stanford.edu/IR-book/'                      # ...
]

And we represent corresponding assessments, as above, as lists of the same size containing relevance values:

In [13]:
my_assessment = [1, 0, 1]
another_assessment = [0, 0, 1]

In order to nicely display URL lists, with or without related assessments, we define a function called `display_results`:

In [14]:
from IPython.display import display, HTML

def display_results(urls, assessment1=None, assessment2=None):
    lines = []
    lines.append('<table>')
    header = '<tr><th>#</th><th>Result URL</th>'
    if (assessment1):
        header += '<th>Assessment 1</th>'
    if (assessment2):
        header += '<th>Assessment 2</th>'
    header += '</tr>'
    lines.append(header)
    i = 0
    for url in urls:
        show_url = url
        if (len(url) > 80):
            show_url = url[:75] + '...'
        line = '<tr><td>{}</td><td><a href="{:s}">{:s}</a></td>'.format(i+1, url, show_url)
        if (assessment1):
            if (assessment1[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        if (assessment2):
            if (assessment2[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        line += '</tr>'
        lines.append(line)
        i = i+1
    lines.append('</table>')
    display( HTML(''.join(lines)) )

We can use this function to display a list of URLs, optionally together with one or two assessment lists:

In [15]:
print("Just a list of URLs:")
display_results(urls)

print("With one assessment:")
display_results(urls, my_assessment)

print("With two assessments:")
display_results(urls, my_assessment, another_assessment)

Just a list of URLs:


#,Result URL
1,https://en.wikipedia.org/wiki/Information_retrieval/
2,http://www.dictionary.com/browse/information
3,https://nlp.stanford.edu/IR-book/


With one assessment:


#,Result URL,Assessment 1
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant
2,http://www.dictionary.com/browse/information,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant


With two assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant,Not relevant
2,http://www.dictionary.com/browse/information,Not relevant,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant,Relevant


Now we are ready to perform an actual evaluation, which will involve a substantial amount of manual work.

---

# Tasks

**Your name:** Aldric de Jacquelin #2711498

### Task 1

Think up and formulate a information need (for example in the field of Computer Science or Medicine) for which you think the answer can be found in scientific publications. On page 152 in the book an example of such an information need is shown: "Information on whether drinking red wine is more effective at reducing the risk of heart attacks than white wine."

**Answer:**information need: Ozempic treatment efficiency of adult patients with diabetes mellitus to improve glycemic control, in combination with metformin (second-line treatment),and sulfonylurea (third-line treatment);


Next, write down specifically what documents have to look like to satisfy your information need. For example if your information need is about finding an overview of different cancer types, you could state that a document would need to list at least ten types of cancer to satisfy your information need (among other criteria). Write this down as a protocol with rules and examples. For example, such a protocol could state that at least three out of five given criteria have to be fulfilled for a document to be considered relevant for the information need, and then specify the criteria. Or your protocol could have the form of a sequence of rules, where each rule lets you either label the document as relevant or not relevant, or proceed with the next rule. Such rules and criteria can, for example, be about the general topic of the paper, the concepts mentioned in it, the covered relations between concepts, the type of publication (research paper, overview paper, etc.), the number of references, the types of contained diagrams, and so on, depending on your specified information need.

**Answer:** protocol for document relevance assessment:

Criteria for Relevance:
-Subject Matter: The document must specifically discuss the use of Ozempic (semaglutide) in the treatment of adult diabetes mellitus.
-Treatment Combination: It must examine the combination therapy of Ozempic with metformin and/or sulfonylurea.
-Outcome Measures: The document should report on specific outcomes related to glycemic control, such as HbA1c levels, fasting plasma glucose, or other relevant biomarkers.
-Study Type: The document should be a peer-reviewed research study, such as a randomized controlled trial (RCT), cohort study, or systematic review/meta-analysis focusing on the treatment efficacy.
-Recency: The document must be recent, ideally published within the last five years, to ensure the information is up-to-date.
-Sample Size and Statistics: The study should have a sufficient sample size for the results to be statistically significant and ideally would report confidence intervals or effect sizes.

Protocol Application Sequence:
-Rule 1: If the document does not pertain to the treatment of adult diabetes mellitus, discard it as irrelevant.
-Rule 2: If the document pertains to diabetes mellitus treatment but does not specifically discuss Ozempic or the combination with metformin and sulfonylurea, discard it as irrelevant.
-Rule 3: If the document discusses the treatment but does not provide specific outcome measures for glycemic control, discard it as irrelevant.
-Rule 4: If the document is not a peer-reviewed study or is outdated, discard it as irrelevant.
-Rule 5: If the document is a peer-reviewed study on the topic with recent data but lacks sufficient sample size or statistical analysis to support the findings, flag it for further consideration but do not consider it fully relevant.
A document is considered relevant if it satisfies all the above rules.

Example of relevance assessment in this case:
A systematic review published 3 years ago analyzing multiple RCTs of Ozempic in combination with metformin and sulfonylurea, reporting HbA1c levels before and after treatment, with a large sample size and clear statistical results, would be considered highly relevant.
in opposite, an older narrative review discussing diabetic treatments in general, without specific data on Ozempic combined with metformin and sulfonylurea, or lacking specific glycemic control outcomes, would be deemed irrelevant.


### Task 2

Formulate a keyword query that represents the information need. For the example on page 152 in the book (see above), the example query "wine AND red AND white AND heart AND attack AND effective" is given. (You don't need to use connectors like "AND", but if you do, make first sure your chosen search engines below actually support them.)

**Answer:** keyword query: (Ozempic OR Semaglutide) AND treatment AND diabetes AND glycemic control AND (mellitus OR metformin OR sulfonylurea)

Then submit your query to **two** of the following academic search engines:

- [Google Scholar](https://scholar.google.com) (all science disciplines)
- [Semantic Scholar](https://www.semanticscholar.org) (all science disciplines)
- [PubMed Search](https://www.ncbi.nlm.nih.gov/pubmed) (Life Sciences / biomedicine)

The right choice of two from the three search engine depends on the topic of your information need. If your information need is in the Life Sciences and biomedicine, it's probably best to include PubMed Search, but otherwise you should pick Google Scholar and Semantic Scholar.

Extract a list of the top 10 URLs of the lists of each of the search engines given the query. To be ensure that your results are reproducible, it is advised to use the private mode of your browser. Try to access the resulting publications. For the publications where that is not possible (because of dead links or because the publication is pay-walled even within the VU network), exclude them from the list and add more publications to the end of your list (that is, append results number 11, then 12, etc. to ensure you have two lists of 10 publications each). In order to deal with paywalls, you should try accessing the articles from the VU network, use
[UBVU Off-Campus
Access](http://www.ub.vu.nl.vu-nl.idm.oclc.org/nl/faciliteiten/toegang-buiten-de-campus/index.aspx), or try to find the respective documents from alternative sources (Google Scholar, for example, is very good at finding free PDFs of articles). If you get fewer than 10 results for one of the search engines, modify the keyword query above to make it more inclusive, and then redo the steps of this task.

Store your two lists of URLs in the form of Python lists as introduced above. Then, use the `display_results` function to nicely display them.

In [21]:
# Create two of the lists below, depending on your chosen engines:

urls_PubMed = [
    'https://pubmed.ncbi.nlm.nih.gov/31305974/',
    'https://pubmed.ncbi.nlm.nih.gov/31305971/',
    'https://pubmed.ncbi.nlm.nih.gov/35194917/',
    'https://pubmed.ncbi.nlm.nih.gov/34706925/',
    'https://pubmed.ncbi.nlm.nih.gov/34305810/',
    'https://pubmed.ncbi.nlm.nih.gov/36050763/',
    'https://pubmed.ncbi.nlm.nih.gov/29049653/',
    'https://pubmed.ncbi.nlm.nih.gov/34248838/',
    'https://pubmed.ncbi.nlm.nih.gov/32987188/',
    'https://pubmed.ncbi.nlm.nih.gov/33964002/'
]
urls_Gscholar = [
    'https://jamanetwork.com/journals/jama/article-abstract/2729339?casa_token=rhDKWigEvHQAAAAA:32qzghsZ7LtTCiQqFjWDHt1CMMC5WdWrL5Ww3TKcLTUfBykIf67p8M1M59IjwaLk5-KgqdMLlGQ',
    'https://www.sciencedirect.com/science/article/pii/S1499267118301096',
    'https://jamanetwork.com/journals/jama/article-abstract/2657376',
    'https://link.springer.com/article/10.1007/s00125-017-4289-0',
    'https://www.thelancet.com/journals/landia/article/PIIS2213-8587(17)30085-2/fulltext',
    'https://diabetesjournals.org/care/article/42/12/2262/36240/Efficacy-Safety-and-Tolerability-of-Oral',
    'https://diabetesjournals.org/care/article/42/12/2272/36229/Oral-Semaglutide-Versus-Empagliflozin-in-Patients',
    'https://www.tandfonline.com/doi/full/10.1080/13696998.2019.1614009',
    'https://dom-pubs.onlinelibrary.wiley.com/doi/full/10.1111/dom.13564',
    'https://www.thelancet.com/article/S2213-8587(17)30092-X/fulltext'
]
# Call display_results here
personal_assessment_PubMed =    [1,1,0,0,1,1,0,1,0,0]
personal_assessment_Gscholar =  [1,1,1,1,1,1,0,1,0,1] 

print("research on PubMed With personal assessments:")
display_results(urls_PubMed, personal_assessment_PubMed)
print("research on Gscholar With personal assessments:")
display_results(urls_Gscholar, personal_assessment_Gscholar)


research on PubMed With personal assessments:


#,Result URL,Assessment 1
1,https://pubmed.ncbi.nlm.nih.gov/31305974/,Relevant
2,https://pubmed.ncbi.nlm.nih.gov/31305971/,Relevant
3,https://pubmed.ncbi.nlm.nih.gov/35194917/,Not relevant
4,https://pubmed.ncbi.nlm.nih.gov/34706925/,Not relevant
5,https://pubmed.ncbi.nlm.nih.gov/34305810/,Relevant
6,https://pubmed.ncbi.nlm.nih.gov/36050763/,Relevant
7,https://pubmed.ncbi.nlm.nih.gov/29049653/,Not relevant
8,https://pubmed.ncbi.nlm.nih.gov/34248838/,Relevant
9,https://pubmed.ncbi.nlm.nih.gov/32987188/,Not relevant
10,https://pubmed.ncbi.nlm.nih.gov/33964002/,Not relevant


research on Gscholar With personal assessments:


#,Result URL,Assessment 1
1,https://jamanetwork.com/journals/jama/article-abstract/2729339?casa_token=r...,Relevant
2,https://www.sciencedirect.com/science/article/pii/S1499267118301096,Relevant
3,https://jamanetwork.com/journals/jama/article-abstract/2657376,Relevant
4,https://link.springer.com/article/10.1007/s00125-017-4289-0,Relevant
5,https://www.thelancet.com/journals/landia/article/PIIS2213-8587(17)30085-2/...,Relevant
6,https://diabetesjournals.org/care/article/42/12/2262/36240/Efficacy-Safety-...,Relevant
7,https://diabetesjournals.org/care/article/42/12/2272/36229/Oral-Semaglutide...,Not relevant
8,https://www.tandfonline.com/doi/full/10.1080/13696998.2019.1614009,Relevant
9,https://dom-pubs.onlinelibrary.wiley.com/doi/full/10.1111/dom.13564,Not relevant
10,https://www.thelancet.com/article/S2213-8587(17)30092-X/fulltext,Relevant


### Task 3

Then, find a fellow student who will **independently**
assess the results as "relevant" or "not relevant" using the protocol that you
have defined above, and also help (at least) one other student for his/her
assessment. Write down their names here:

**Name of the student who assesses my results:** Mohammad Reshadati

**Name of the student who I help to assess his/her results:** Mohammad Reshadati

Show to the other assessor everything you have written down above for Tasks 1 and 2 (and you might also want to give him/her the PDFs you got for these papers to simplify the process).

You as assessors need to stick to the protocol you made in Task 1 and should not discuss with each other, especially when you doubt whether a result is relevant or not. Write down your assessments as lists of relevance values, as introduced above, and make sure they correctly map to the URLs by displaying them together with the `display_results` function.

To avoid problems with extreme results, mark in each list at least one paper as 'relevant' and at least one paper as 'not relevant'. That is, if all papers seem relevant, mark the one that seems least relevant 'not relevant', and conversely, if none of the papers seem relevant, mark the one that seems a bit more relevant than the others as 'relevant'.

In [17]:
# 0 = not relevant; 1 = relevant

# You only need to create 4 of the following 6 lists, again depending on which search engines you chose.

# Assessment 1 is from you:

personal_assessment_PubMed =    [1,1,0,0,1,1,0,1,0,0]
personal_assessment_Gscholar =  [1,1,1,1,1,1,0,1,0,1] 
# Assessment 2 is from another assessor (don't show him/her your own assessment!):

assessment2_PubMed =    [1,1,0,1,1,0,1,1,0,0]
assessment2_Gscholar =  [1,1,0,1,1,0,1,1,0,1]

# Call display_results here

urls_PubMed = [
    'https://pubmed.ncbi.nlm.nih.gov/31305974/',
    'https://pubmed.ncbi.nlm.nih.gov/31305971/',
    'https://pubmed.ncbi.nlm.nih.gov/35194917/',
    'https://pubmed.ncbi.nlm.nih.gov/34706925/',
    'https://pubmed.ncbi.nlm.nih.gov/34305810/',
    'https://pubmed.ncbi.nlm.nih.gov/36050763/',
    'https://pubmed.ncbi.nlm.nih.gov/29049653/',
    'https://pubmed.ncbi.nlm.nih.gov/34248838/',
    'https://pubmed.ncbi.nlm.nih.gov/32987188/',
    'https://pubmed.ncbi.nlm.nih.gov/33964002/'
]
urls_Gscholar = [
    'https://jamanetwork.com/journals/jama/article-abstract/2729339?casa_token=rhDKWigEvHQAAAAA:32qzghsZ7LtTCiQqFjWDHt1CMMC5WdWrL5Ww3TKcLTUfBykIf67p8M1M59IjwaLk5-KgqdMLlGQ',
    'https://www.sciencedirect.com/science/article/pii/S1499267118301096',
    'https://jamanetwork.com/journals/jama/article-abstract/2657376',
    'https://link.springer.com/article/10.1007/s00125-017-4289-0',
    'https://www.thelancet.com/journals/landia/article/PIIS2213-8587(17)30085-2/fulltext',
    'https://diabetesjournals.org/care/article/42/12/2262/36240/Efficacy-Safety-and-Tolerability-of-Oral',
    'https://diabetesjournals.org/care/article/42/12/2272/36229/Oral-Semaglutide-Versus-Empagliflozin-in-Patients',
    'https://www.tandfonline.com/doi/full/10.1080/13696998.2019.1614009',
    'https://dom-pubs.onlinelibrary.wiley.com/doi/full/10.1111/dom.13564',
    'https://www.thelancet.com/article/S2213-8587(17)30092-X/fulltext'
]
# Call display_results here
print("research on PubMed With personal assessments:")
display_results(urls_PubMed, personal_assessment_PubMed, assessment2_PubMed)
print("research on Gscholar With personal assessments:")
display_results(urls_Gscholar, personal_assessment_Gscholar, assessment2_Gscholar)


research on PubMed With personal assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://pubmed.ncbi.nlm.nih.gov/31305974/,Relevant,Relevant
2,https://pubmed.ncbi.nlm.nih.gov/31305971/,Relevant,Relevant
3,https://pubmed.ncbi.nlm.nih.gov/35194917/,Not relevant,Not relevant
4,https://pubmed.ncbi.nlm.nih.gov/34706925/,Not relevant,Relevant
5,https://pubmed.ncbi.nlm.nih.gov/34305810/,Relevant,Relevant
6,https://pubmed.ncbi.nlm.nih.gov/36050763/,Relevant,Not relevant
7,https://pubmed.ncbi.nlm.nih.gov/29049653/,Not relevant,Relevant
8,https://pubmed.ncbi.nlm.nih.gov/34248838/,Relevant,Relevant
9,https://pubmed.ncbi.nlm.nih.gov/32987188/,Not relevant,Not relevant
10,https://pubmed.ncbi.nlm.nih.gov/33964002/,Not relevant,Not relevant


research on Gscholar With personal assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://jamanetwork.com/journals/jama/article-abstract/2729339?casa_token=r...,Relevant,Relevant
2,https://www.sciencedirect.com/science/article/pii/S1499267118301096,Relevant,Relevant
3,https://jamanetwork.com/journals/jama/article-abstract/2657376,Relevant,Not relevant
4,https://link.springer.com/article/10.1007/s00125-017-4289-0,Relevant,Relevant
5,https://www.thelancet.com/journals/landia/article/PIIS2213-8587(17)30085-2/...,Relevant,Relevant
6,https://diabetesjournals.org/care/article/42/12/2262/36240/Efficacy-Safety-...,Relevant,Not relevant
7,https://diabetesjournals.org/care/article/42/12/2272/36229/Oral-Semaglutide...,Not relevant,Relevant
8,https://www.tandfonline.com/doi/full/10.1080/13696998.2019.1614009,Relevant,Relevant
9,https://dom-pubs.onlinelibrary.wiley.com/doi/full/10.1111/dom.13564,Not relevant,Not relevant
10,https://www.thelancet.com/article/S2213-8587(17)30092-X/fulltext,Relevant,Relevant


### Task 4

Compute Cohen's kappa to quantify how much the two assessors agreed. Use the function `cohen_kappa_score` demonstrated above to calculate two times the inter-annotator agreement (once for each of the two search engines), and print out the results.

In [18]:
# Add your code here:

kappa_google = cohen_kappa_score(personal_assessment_Gscholar,assessment2_Gscholar)
kappa_pubmed = cohen_kappa_score(personal_assessment_PubMed,assessment2_PubMed)

print("Kappa for Google Scholar:", kappa_google)
print("Kappa for PubMed:", kappa_pubmed)

Kappa for Google Scholar: 0.21052631578947367
Kappa for PubMed: 0.4


Explain whether the agreement can be considered high or not, based on the interpretation table on [this Wikipedia page](https://en.wikipedia.org/wiki/Fleiss'_kappa#Interpretation) (this Wikipedia page is about a different type of kappa but the interpretation table can also be used for Cohen's kappa).

In [19]:
#Answer:
#Kappa for Google Scholar ~ 0.21: considered as slight to fair agreement.
#Kappa for PubMed = 0.4 : falls into the moderate agreement category.

#Answer:

Kappa for Google Scholar ~ 0.21: considered as slight to fair agreement.

Kappa for PubMed = 0.4 : falls into the moderate agreement category.

### Task 5

Define a function called `precision_at_n` that calculates Precision@n as described in the lecture slides, which takes as input an assessment list and a value for _n_ and returns the respective Precision@n value. Run this function to calculate Precision@10 (that is, n=10) on all four assessments (two assessors and two search engines).

In [20]:
def precision_at_n(assessment_x, n):
    relevant_files = sum(assessment_x[:n])
    precision_n = relevant_files / n
    return precision_n

precision_at_10_personal_PubMed = precision_at_n(personal_assessment_PubMed, 10)
precision_at_10_assessment2_PubMed = precision_at_n(assessment2_PubMed, 10)
precision_at_10_personal_Gscholar = precision_at_n(personal_assessment_Gscholar, 10)
precision_at_10_assessment2_Gscholar = precision_at_n(assessment2_Gscholar, 10)
print("Precision@10 for personal assessment on PubMed:", precision_at_10_personal_PubMed)
print("Precision@10 for external assessment2 on PubMed:", precision_at_10_assessment2_PubMed)
print("Precision@10 for personal assessment on Google Scholar:", precision_at_10_personal_Gscholar)
print("Precision@10 for external assessment2 on Google Scholar:", precision_at_10_assessment2_Gscholar)


Precision@10 for personal assessment on PubMed: 0.5
Precision@10 for external assessment2 on PubMed: 0.6
Precision@10 for personal assessment on Google Scholar: 0.8
Precision@10 for external assessment2 on Google Scholar: 0.7


Explain what these specific Precision@10 results tell us (or don't tell us) about the quality of the two search engines for your particular information need. You can also refer to the results of Task 4 if necessary.

Personal assessment on PubMed =0.5: Half of the top 10 results were relevant to the specified information need. This suggests moderate relevance of the search results.

External assessment on PubMed=0.6: A slightly higher relevance than the personal assessment, indicating that 60% of the top 10 results were relevant.

Personal assessment on Google Scholar = 0.8: A high proportion (80%) of the top10 results were relevant, indicating that Google Scholar provided more relevant results for the query/information need.

External Assessment on Google Scholar = 0.7: This is lower than the personal assessment still suggests a high relevance of the search results.

Precision@10 results imply that for this specific information need, Google Scholar returned a higher number of relevant documents in its top 10 results compared to PubMed, according to the assessor's and likewise an external evaluator's assessments. 
However, it doesn't account for the total number of relevant documents in the entire dataset (recall), nor does it consider the rank of the relevant documents beyond the top 10 (which could be captured by average precision or discounted cumulative gain).

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.